<a href="https://colab.research.google.com/github/m-hess/deep-learning-training/blob/master/TensorflowQuickStartTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setups

In [0]:
import tensorflow as tf

Load and prep the [MNIST dataset](http://yann.lecun.com/exdb/mnist/)

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


Build the [`tf.keras.Sequential`](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) model by stacking layers. Choose an optimizedr and loss function for training.

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of ["logits"](https://developers.google.com/machine-learning/glossary#logits) or ["log-odds"](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [45]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.26915142,  0.39601508, -0.42259416,  0.21792316,  0.54816383,
        -0.4814065 ,  0.07629816,  0.8714194 ,  0.15200478, -0.30125427]],
      dtype=float32)

The [`tf.nn.softmax`](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to "probabilities" for each class:

In [46]:
tf.nn.softmax(predictions).numpy()

array([[0.10542338, 0.11968318, 0.05278562, 0.10015872, 0.13935113,
        0.0497707 , 0.08693244, 0.19253008, 0.09376933, 0.05959542]],
      dtype=float32)

The [`losses.SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class. 

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3)`

In [48]:
loss_fn(y_train[:1], predictions).numpy()

3.0003288

In [0]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The [`Model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method adjusts the model parameters to minimize the loss:

In [50]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2942 - accuracy: 0.9140
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1411 - accuracy: 0.9578
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1051 - accuracy: 0.9682
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0865 - accuracy: 0.9726
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0722 - accuracy: 0.9775


The [`Model.evaluate`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the model's performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [51]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - loss: 0.0739 - accuracy: 0.9764


[0.07394060492515564, 0.9764000177383423]

The image classifier is now trained to ~98% accuracy!

If you want the model to return a probability, you can wrap the trained model and attach the softmax to it:

In [0]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [53]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[7.67181291e-07, 5.30797051e-09, 1.33525318e-05, 2.75521423e-03,
        3.16652343e-10, 8.86946978e-08, 1.87186920e-12, 9.97194290e-01,
        5.25724226e-06, 3.09436182e-05],
       [9.20866647e-11, 3.99787168e-05, 9.99954820e-01, 4.33398873e-06,
        1.41211726e-14, 7.92527374e-07, 7.09059123e-09, 1.85649239e-12,
        8.67249597e-08, 5.96282951e-13],
       [4.18793888e-07, 9.99335825e-01, 1.11148918e-04, 9.33888077e-06,
        8.71135853e-05, 8.87261592e-07, 1.39553440e-05, 3.16322898e-04,
        1.18560136e-04, 6.35685137e-06],
       [9.99944210e-01, 5.67153080e-09, 1.91044437e-05, 6.18715740e-06,
        1.58360631e-08, 5.22930350e-06, 1.97278928e-06, 9.20665116e-06,
        2.22505374e-08, 1.39982531e-05],
       [5.14070962e-07, 7.94454669e-10, 3.09606708e-06, 1.32405842e-06,
        9.99180257e-01, 1.58301901e-07, 3.53815523e-07, 3.74676129e-05,
        6.27007921e-07, 7.76209694e-04]], dtype=float32)>